In [1]:
import pandas as pd 
import numpy as np
from pathlib import Path
from shapely.geometry import Polygon, Point
from shapely.ops import transform
from functools import partial
import pyproj
from classLib_forFlightplans import *
import random
import operator
from functools import reduce

In [2]:
air=pd.read_json("../airways.json")


def filter_airways(x1,y1,x2,y2,x3,y3,x4,y4,airway):
    filtered_airways = pd.DataFrame()

    boundary_coordinates = ([x1,y1],[x2,y2],[x3,y3],[x4,y4])

    poly = Polygon(boundary_coordinates)
    for i in range(len(airway)):
        for j in range(len(airway.segments[i])):
            point1 = Point([float(airway.segments[i][j]['lat']),float(airway.segments[i][j]['lon'])])
        # point2 = Point([float(airway.segments[i][1]['lat']),float(airway.segments[i][1]['lon'])])
            if poly.contains(point1)== True :
            # and poly.contains(point2)== True
                filtered_airways=filtered_airways.append(airway.loc[i])
    filtered_airways = filtered_airways.drop_duplicates(subset=['name'])
    return filtered_airways

In [3]:
filteraws = filter_airways(0,100,0,110,10,110,10,100,air)
filteraws = filteraws.reset_index(drop=True)
filteraws.head(), len(filteraws)

(   name                                           segments
 0  A224  [{'name': 'JOHOR BAHRU DVOR/DME (VJB)', 'lat':...
 1  A457  [{'name': 'JOHOR BAHRU DVOR/DME (VJB)', 'lat':...
 2  A464  [{'name': 'ARAMA (50 DME SJ)(Delegated airspac...
 3  A576  [{'name': 'REKOP (50 DME SJ)(delegated airspac...
 4  B338  [{'name': 'MERSING DVOR/DME (VMR)', 'lat': '2....,
 57)

In [4]:
"""This snippet adds transformed x and y columns to the airways file"""
a=np.array([0,100])
a_ = np.array([a[1],a[0]])
b=  np.array([0,110])
b_ = np.array([b[1],b[0]])
c =np.array([10,110])
c_ = np.array([c[1],c[0]])
d=  np.array([10,100])
d_ = np.array([d[1],d[0]])

horizontal_len= b[1]-a[1]
vertical_len = d[0]-a[0]

dict1=[]

for i in range(len(filteraws)):
    # print(i)
    d1= []
    for j in range(len(filteraws.segments[i])):
        # print(j)
        P = np.array([float(filteraws.segments[i][j]['lon']),float(filteraws.segments[i][j]['lat'])])
        # print(P)
        # print("^^")
        # vert_d = np.linalg.norm(np.cross(b_-a_,P-a_))/np.linalg.norm(b_-a_)
        # horiz_d = np.linalg.norm(np.cross(d_-a_,P-a_))/np.linalg.norm(d_-a_)
        vert_d = P[1]-a_[1]
        horiz_d = P[0] -a_[0]
        airdict1={
                # "name":filteraws.name[i],        
                # "segments":[{"name":filteraws.segments[i][j]['name'],
                "name":filteraws.segments[i][j]['name'],
                "lat":filteraws.segments[i][j]['lat'],
                "lon":filteraws.segments[i][j]['lon'],
                "latdms":filteraws.segments[i][j]['latdms'],
                "londms":filteraws.segments[i][j]['londms'],
                "transf_x": (900/horizontal_len)*horiz_d,
                "transf_y": (900/vertical_len)*vert_d
                }
        d1.append(airdict1)
    # dict2={"segments":d1}
    dict3 ={"name":filteraws.name[i], 
            "segments":d1}
    # print(dict3)
    dict1.append(dict3)
        
df = pd.DataFrame(dict1)

In [5]:
datalist=[]
for i in range(10):
    waypointList = []
    for i in range(len(df)):
        for j in range(len(df.segments[i])):
            waypointList.append(WayPoint(df.segments[i][j]['name'],[df.segments[i][j]['transf_x'],900-df.segments[i][j]['transf_y']]))
            
    len(waypointList),len(df.segments)

    airwayList=[]
    a=0
    for i in range(len(df.segments)):
        x=len(df.segments[i])
        # print(x)
        for j in range(a,len(waypointList[:a+x])-1):
            li=[waypointList[j],waypointList[j+1]]
            # print(li)
            airwayList.append(Airways('aw'+ str(j),li[0],li[1]))
        a+=x

    route1 = ['MELAS (VVTS/WSJC FIR BDRY)','MABLI','MUMSO','MABAL','KILOT','KIBOL WSJC/WMFC FIR BDRY','PEKLA']
    route2 = ['ENREP','NOPAT','DAMOG','SUSAR','MUMSO','MABAL','KILOT','KIBOL WSJC/WMFC FIR BDRY','PEKLA']


    r1=[]
    for i in (route1):
        for j in range(len(waypointList)):
            if i == waypointList[j].name:
                r1.append(waypointList[j])
                break
    r2=[]
    for i in (route2):
        for j in range(len(waypointList)):
            if i == waypointList[j].name:
                r2.append(waypointList[j])
                break

    ac_paths=[r1,r2]
    aircraft_type=['medium','heavy','super_heavy']

    ac_per_scenario = 2 
    aircraftList=[]
    aircraft_name=[]
    routelist=[]
    for n in range(1,ac_per_scenario+1):
        aircraft_name.append('ac'+str(n))
    for i in range(ac_per_scenario):
        route = np.random.choice(ac_paths)
        
        d=[20,30,40,50,60,80,100,120,140,150,170,190,210,220] #offset,
        aircraft = Aircraft(str(aircraft_name[i]), route[0] ,route[-1], route,  np.random.choice(d)) #this will create multiple scenarios
        aircraftList.append(aircraft)
        routelist.append(route)


    aircraft_paths_x = []
    aircraft_paths_y = []
    path_waypoints   = []
    for i in range(len(routelist)):
        x_list=[]
        y_list=[]
        wps_list=[]
        for j in range(len((routelist[i]))):
            wps_list.append(routelist[i][j].name)
            x=routelist[i][j].x
            y=routelist[i][j].y
            x_list.append(x)
            y_list.append(y)
        aircraft_paths_x.append(x_list)
        aircraft_paths_y.append(y_list)
        path_waypoints.append(wps_list)


    x_dirlist=[]
    y_dirlist=[]
    for i in range(len(aircraftList)):
        x_dir =[]
        y_dir =[]
        for j in range(len(aircraftList[i].route)-1):
            startpoint = np.array([aircraftList[i].route[j].x, aircraftList[i].route[j].y])
            endpoint = np.array([aircraftList[i].route[j+1].x, aircraftList[i].route[j+1].y])
            segment = np.linalg.norm(startpoint - endpoint)
            xdir = (endpoint[0] - startpoint[0])/segment
            ydir = (endpoint[1] - startpoint[1])/segment
            x_dir.append(xdir)
            y_dir.append(ydir)
        x_dirlist.append(x_dir)
        y_dirlist.append(y_dir)


    vel=0.25
    timelist=[]
    x_loclist=[]
    y_loclist=[]
    total_time=[]
    for i in range(len(aircraftList)):
        x_loc=[]
        y_loc=[]
        time=[]
        section_time=[]
        for j in range(len(aircraftList[i].route)-1):
            startpoint = np.array([aircraftList[i].route[j].x, aircraftList[i].route[j].y])
            endpoint = np.array([aircraftList[i].route[j+1].x, aircraftList[i].route[j+1].y])
            distance = np.linalg.norm(startpoint - endpoint)
            totaltime = distance/vel    #(time per kilometer in seconds, bwtween two waypoints)
            
            for t in range(int(totaltime)):
                x_point = startpoint[0] + t*x_dirlist[i][j]*vel
                y_point = startpoint[1] + t*y_dirlist[i][j]*vel
                x_loc.append(x_point)
                y_loc.append(y_point)
                time.append(t)
                
            section_time.append(totaltime)
        # print(len(section_time))
        timelist.append(time)
        x_loclist.append(x_loc)
        y_loclist.append(y_loc)
        total_time.append(section_time)

        
    #repeating the directins vectors based on total time
    all_dirs_x=[]
    all_dirs_y=[]
    for i in range(len(total_time)): #2
        x_list=[]
        y_list=[]
        for j in range(len(total_time[i])):
            x = np.repeat(x_dirlist[i][j], int(total_time[i][j]))
            y = np.repeat(y_dirlist[i], int(total_time[i][j]))
            # print(len(x))
            x= x.tolist()
            y= y.tolist()
            # print(len(x))
            # print(len(y[]))
            x_list.append(x)
            y_list.append(y)
        
        all_dirs_x.append(x_list)
        all_dirs_y.append(y_list)

    x_directions_flat=[]
    y_directions_flat=[]
    for i in range(len(x_dirlist)): #2
        x_l= all_dirs_x[i]
        x_flat = reduce(operator.concat,x_l)
        # print(x_flat)
        x_directions_flat.append(x_flat)

        y_l = all_dirs_y[i]
        y_flat = reduce(operator.concat,y_l)
        y_directions_flat.append(y_flat)

    # len(x_directions_flat[0])

    dict4={}
    for i in range(len(waypointList)):
        waydict={waypointList[i].name:{
            'x':waypointList[i].x,
            'y':waypointList[i].y}}
        dict4.update(waydict)


    dict5={}
    for i in range(len(airwayList)):
        airwaydict= {airwayList[i].name:{
            'start':airwayList[i].start_wp,
            'end' : airwayList[i].end_wp}}
        dict5.update(airwaydict)


    dict6={}
    for i in range(len(aircraftList)):
        aircraftdict= {
            aircraftList[i].name:{
            'x': aircraftList[i].offset_x,
            'y':aircraftList[i].offset_y,
            'dir_x': x_dirlist[i][0],
            'dir_y': y_dirlist[i][0],
            'x_dirs':x_directions_flat[i],
            'y_dirs':y_directions_flat[i],
            'type': np.random.choice(aircraft_type),
            'altitude':360,
            'path_waypoints': path_waypoints[i],
            'x_path' : x_loclist[i],
            'y_path' : y_loclist[i],
            'timestep': timelist[i]
            }
            }
        dict6.update(aircraftdict)


    df2=pd.read_json('SgSectors_LonLat.json')


    """the sqaure region with 'a' as the bottom left point (moving anticlock abcd)
        a,b,c,d= [lon,lat] format"""
    a = np.array([100,0])
    b = np.array([110,0])
    c = np.array([110,10])
    d = np.array([100,10])

    horizontal_length  = b[0]-a[0]
    vertical_length = d[1]-a[1]
    keylist=['sector1','sector2','sector3','sector4','sector5','sector6','sector7','sector8']
    l1=[[],[],[],[],[],[],[],[]]
    for i in range(len(keylist)):
        for j in range(len(df2.Sectors[0][keylist[i]])):
            P= np.array([df2.Sectors[0][keylist[i]][j][0],df2.Sectors[0][keylist[i]][j][1]])
            # print(P)
            vert_d = P[1]-a[1]
            horiz_d= P[0]-a[0]
            l1[i].append([(900/horizontal_length)*horiz_d,900-(900/vertical_length)*vert_d])



    data_dict={'waypoints':dict4, 'airways':dict5,'aircrafts':dict6, 
            'sectors':{
                        'sector1':{'type': 'secondary', 'boundary': l1[0]},
                        'sector2':{'type': 'secondary', 'boundary': l1[1]},
                        'sector3':{'type': 'secondary', 'boundary': l1[2]},
                        'sector4':{'type': 'secondary', 'boundary': l1[3]},
                        'sector5':{'type': 'secondary',  'boundary': l1[4]},
                        'sector6':{'type': 'primary', 'boundary':  l1[5]},
                        'sector7':{'type':'secondary', 'boundary': l1[6]},
                        'sector8':{'type':'secondary' ,'boundary':l1[7]}
                        }}

    datalist.append(data_dict)

import json
with open('scenario_data/25August_FlightPlanTest.json', 'w') as json_file:
    json.dump(datalist, json_file) 


In [ ]:
r1,r2

In [ ]:
aircraftList[0].start_wp_x,aircraftList[0].offset_x

In [ ]:
a=np.random.choice(ac_paths)
a[0].name,a[-1].name

In [ ]:
aircraftList[0].route[0].x

In [ ]:
len(x_dirlist)

In [ ]:
len(total_time[0]),len(x_dirlist[0])

In [ ]:
m=[[1,2],[34,2],[12,23,34,3]]
mm = reduce(operator.concat,m)
mm

In [ ]:
aircraftList[1].start_wp_x ,aircraftList[1].offset_x

In [ ]:

ac_paths[0]

In [ ]:
""""here, the list aircraft_paths contains the waypoint lists that an aircraft will move on"""
# aircraft_paths=[] 
# b=0
# for i in range(len(df.segments)):
#     x= len(df.segments[i])
#     segment=[]
#     for j in range(b,len(waypointList[:b+x])):
#         li=waypointList[j]
#         segment.append(li)
#     aircraft_paths.append(segment)
#     b+=x

# aircraft_paths[1]

In [ ]:
len

In [ ]:
# #change things accorgdign to ac_paths
# aircraft_paths_x = []
# aircraft_paths_y = []
# path_waypoints   = []
# for i in range(len(ac_paths)):
#     x_list=[]
#     y_list=[]
#     wps_list=[]
#     for j in range(len((ac_paths[i]))):
#         wps_list.append(ac_paths[i][j].name)
#         x=ac_paths[i][j].x
#         y=ac_paths[i][j].y
#         x_list.append(x)
#         y_list.append(y)
#     aircraft_paths_x.append(x_list)
#     aircraft_paths_y.append(y_list)
#     path_waypoints.append(wps_list)
# path_waypoints[0]


In [ ]:
#change things accorgdign to ac_paths


In [ ]:
routelist[0]

In [ ]:

# ac_pathlongerthan2= []
# for i in range(len(a_paths)):
#     if len(aircraft_paths[i])>2:
#         ac_pathlongerthan2.append(aircraft_paths[i])
# len(ac_pathlongerthan2)


    


In [ ]:
len(routelist),len(aircraftList)